In [1]:
import pandas as pd

In [13]:
df_sent = pd.read_parquet('../sentiment_analysis_2.parquet', columns= ['item_id', 'sentiment_analysis_2', 'recommend', 'Año'])
df_steam = pd.read_parquet('../CleanData/steam_games.parquet', columns= ['id', 'developer'])
df = df_sent.merge(df_steam, how='left', left_on='item_id', right_on='id')
df.drop(columns=['item_id', 'id'], inplace=True)
df.dropna(inplace=True)
df = df.groupby(['Año', 'developer']).agg({'sentiment_analysis_2': (lambda x: (x == 2).sum()), 'recommend': 'sum'})
df['rating'] = df['sentiment_analysis_2'] + df['recommend']
df.drop(columns=['sentiment_analysis_2', 'recommend'], inplace=True)
df.reset_index(inplace=True)
df.head()

,Año,developer,rating
0,2010,1C Company,2
1,2010,1C: Maddox Games,1
2,2010,Amanita Design,1
3,2010,BioWare,2
4,2010,"BioWare,Aspyr (Mac)",2
...,...,...,...
3770,2015,webgames,1
3771,2015,xXarabongXx,1
3772,2015,△○□× (Miwashiba),2
3773,2015,"インレ,Inre",4


In [14]:
df.to_parquet('../CleanData/best_developer_year.parquet')

In [20]:
def best_developer_year(anio: int):

    try:
        anio = int(anio)
    except Exception as e:
        return {f'Error {e}': 'Debe insertar un número entero.'}
    
    df = pd.read_parquet('../CleanData/best_developer_year.parquet')
    anios = list(df['Año'].unique())
    anios = [int(x) for x in anios]
    
    df = df[df['Año'].isin([anio])]
    if len(df) == 0:
        del df
        return {'Mensaje': f'No hay registros del año {anio}',
                'Los años disponibles son:': anios}

    df = df.sort_values('rating', ascending=False).reset_index()
    
    result = {
                'Puesto 1': df.loc[0, 'developer'],
                'Puesto 2': df.loc[1, 'developer'],
                'Puesto 3': df.loc[2, 'developer']
            }
    del df
    return result

In [21]:
best_developer_year(6447)

{'Mensaje': 'No hay registros del año 6447',
 'Los años disponibles son:': [2010, 2011, 2012, 2013, 2014, 2015]}